In [ ]:
# | default_exp _application.tester

In [ ]:
# | export

import asyncio
import inspect
from contextlib import asynccontextmanager
from typing import *

from fastcore.basics import patch
from fastcore.meta import delegates
from pydantic import BaseModel

from fastkafka._application.app import FastKafka
from fastkafka._testing.local_broker import LocalKafkaBroker

In [ ]:
import pytest
from pydantic import Field

from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(bootstrap_servers="localhost:9092")


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

## Fastkafka Tester class

In [ ]:
# | export


class Tester(FastKafka):
    @delegates(LocalKafkaBroker.__init__)  # type: ignore
    def __init__(
        self,
        app: Union[FastKafka, List[FastKafka]],
        *,
        broker: Optional[LocalKafkaBroker] = None,
        **kwargs: Any,
    ):
        """Mirror-like object for testing a FastFafka application

        Can be used as context manager

        """
        self.apps = app if isinstance(app, list) else [app]
        super().__init__(
            bootstrap_servers=self.apps[0]._kafka_config["bootstrap_servers"]
        )
        self.create_mirrors()

        if broker is None:
            topics = set().union(*(app.get_topics() for app in self.apps))
            kwargs["topics"] = topics
            self.broker = LocalKafkaBroker(**kwargs)
        else:
            self.broker = broker

    async def startup(self) -> None:
        """Starts the Tester"""
        for app in self.apps:
            app.create_mocks()
            await app.startup()
        self.create_mocks()
        await super().startup()
        await asyncio.sleep(3)

    async def shutdown(self) -> None:
        """Shuts down the Tester"""
        await super().shutdown()
        for app in self.apps[::-1]:
            await app.shutdown()

    def create_mirrors(self) -> None:
        pass

    @asynccontextmanager
    async def _create_ctx(self) -> AsyncGenerator["Tester", None]:
        bootstrap_server = await self.broker._start()
        try:
            self.set_bootstrap_servers(bootstrap_servers=bootstrap_server)
            for app in self.apps:
                app.set_bootstrap_servers(bootstrap_server)
            await self.startup()
            try:
                yield self
            finally:
                await self.shutdown()
        finally:
            await self.broker._stop()

    async def __aenter__(self) -> "Tester":
        self._ctx = self._create_ctx()
        return await self._ctx.__aenter__()

    async def __aexit__(self, *args: Any) -> None:
        await self._ctx.__aexit__(*args)


Tester.__module__ = "fastkafka.testing"

In [ ]:
for _ in range(2):
    with pytest.raises(RuntimeError) as e:
        async with Tester(app) as tester:
            assert tester.broker.is_started
            assert tester.is_started
            raise RuntimeError("ok")

    assert not tester.broker.is_started
    assert not tester.is_started

self._producers_store={}
[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': '127.0.0.1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka

In [ ]:
tester = Tester(app, zookeeper_port=9998, listener_port=9788)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

self._producers_store={'preprocessed_signals': (<function to_preprocessed_signals>, None, {})}
[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9788
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': '127.0.0.1:9788', 'auto_offset_reset': 'earliest', 'max_poll_rec

In [ ]:
# | export


def mirror_producer(topic: str, producer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(producer_f).return_annotation

    async def skeleton_func(msg: BaseModel) -> None:
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "on_" + topic

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig  # type: ignore

    return mirror_func

In [ ]:
@app.produces()
def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
def some_log(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, _) in app._producers_store.items():
    mirror = mirror_producer(topic, producer_f)
    assert mirror.__name__ == "on_" + topic
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export


def mirror_consumer(topic: str, consumer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    async def skeleton_func(msg: BaseModel) -> BaseModel:
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "to_" + topic

    # adjust arg and return val
    sig = sig.replace(parameters=[msg_type], return_annotation=msg_type.annotation)

    mirror_func.__signature__ = sig  # type: ignore
    return mirror_func

In [ ]:
for topic, (consumer_f, _) in app._consumers_store.items():
    mirror = mirror_consumer(topic, consumer_f)
    assert mirror.__name__ == "to_" + topic
    assert inspect.signature(mirror).return_annotation == TestMsg, inspect.signature(
        mirror
    ).return_annotation
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export


@patch  # type: ignore
def create_mirrors(self: Tester):
    for app in self.apps:
        for topic, (consumer_f, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(topic, consumer_f)
            mirror_f = self.produces()(mirror_f)  # type: ignore
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, _) in app._producers_store.items():
            mirror_f = mirror_producer(topic, producer_f)
            mirror_f = self.consumes()(mirror_f)  # type: ignore
            setattr(self, mirror_f.__name__, mirror_f)

In [ ]:
async with Tester(app, zookeeper_port=9998, listener_port=9788) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

self._producers_store={'preprocessed_signals': (<function mirror_consumer.<locals>.skeleton_func>, None, {})}
[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9788
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Star